In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime
import json
from string import digits
import re

url = "https://www.cnn.com/"
html = urllib.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

#some code to grab the relevant chunk of html containing articles
scripts = list(soup.find_all("script"))
n=0
for script in scripts:
    if n==0:
        longest_script = script
        n=1
    else:
        if len(str(script)) > len(str(longest_script)):
            longest_script = script
        
cnn_content = longest_script.string

#distilling down to the list of dictionaries containing article content
cnn_list = []
for item in cnn_content.split('{"articleList":'):
    for split in item.split(']}'):
        cnn_list.append(split)

cnn_articles = []        
for stuff in cnn_list:
    if stuff[:8] == '[{"uri":':
        cnn_articles.append(stuff.encode("utf-8"))

articles = cnn_articles[0].replace("'[","") + "]"

article_data = json.loads(articles)

data = []

for article in article_data:
    article_dict = dict(article)
    data.append([article_dict['headline'],article_dict['description'],article_dict['uri'].encode("utf-8"),article_dict['duration'],article_dict['layout'],article_dict['thumbnail']])
    
df = pd.DataFrame(data,columns=['headline','description','url','duration','layout','thumbnail'])

df['date'] = datetime.today().strftime('%Y-%m-%d')
df['year'] = datetime.today().year
df['month'] = datetime.today().month
df['day'] = datetime.today().day
df['hour'] = datetime.today().hour
df['minute'] = datetime.today().minute

sort_ranks = []

for index, record in df.iterrows():
    if record['headline'][:8] == '<strong>':
        sort_ranks.append(1)
    else:
        sort_ranks.append(sort_ranks[-1]+1)
        
df['sort_rank'] = sort_ranks

article_types = []

for index, record in df.iterrows():
    if record['duration'] <> '':
        article_types.append(record['url'].strip('/videos/').split('/')[0])
    else:
        article_types.append(record['url'].translate(None, digits).strip('////').split('/')[0])

df['article_type'] = article_types

#harvesting article text
from selenium import webdriver
from time import sleep

article_text = []
#work comp
#driver = webdriver.Chrome("C:\Users\dbell\Data Science [Github]/chromedriver.exe")

#home comp
driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in df.url:
    driver.get('https://www.cnn.com' + url)
    sleep(1) #wait 1 second
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    article_text.append(soup.find_all(class_=re.compile("paragraph")))

driver.close()    
df['article_text'] = article_text

cnn_articles = pd.read_csv('cnn.csv')
print "cnn articles current shape:"
print cnn_articles.shape[0]
print ""
conc = pd.concat([cnn_articles,df],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('cnn.csv', encoding = 'utf-8')
print "cnn new shape:"
print conc.shape[0]

C:\Users\Duncan\Anaconda2\lib\site-packages\bs4\__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


cnn articles current shape:
56

cnn new shape:
103


In [2]:
conc

,headline,description,url,duration,layout,thumbnail,date,year,month,day,hour,minute,sort_rank,article_type,article_text
0,<strong>The donor's name was also stripped fro...,The university also stripped the donor's name ...,/2019/06/07/us/university-alabama-abortion-don...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190607160333-...,2019-06-07,2019,6,7,19,7,1,us,"[<div class=""el__leafmedia el__leafmedia--sour..."
1,Opinion: Abortion law holds Alabama's tangled ...,"Some years ago, with the Confederate flag <a h...",/2019/05/18/opinions/alabama-abortion-law-stat...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190515195336-...,2019-06-07,2019,6,7,19,7,2,opinions,"[<p class=""zn-body__paragraph""><q class=""el-ed..."
2,North Carolina's GOP-led House fails to overri...,North Carolina's Republican-led House of Repre...,/2019/06/05/politics/nc-house-did-not-override...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/170626110435-...,2019-06-07,2019,6,7,19,7,3,politics,"[<div class=""el__leafmedia el__leafmedia--sour..."
3,Biden reverses course to support repeal of Hyd...,Former Vice President Joe Biden says he now wa...,/videos/politics/2019/06/07/joe-biden-hyde-ame...,02:00,NaN,//cdn.cnn.com/cnnnext/dam/assets/190606180137-...,2019-06-07,2019,6,7,19,7,4,politics,[]
4,Inside Joe Biden's abortion stance reversal,"As Joe Biden prepared to <a href=""https://www....",/2019/06/07/politics/joe-biden-hyde-amendment-...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190606180137-...,2019-06-07,2019,6,7,19,7,5,politics,"[<div class=""el__leafmedia el__leafmedia--sour..."
5,CNN Poll: Abortion growing in importance for v...,Three-in-10 Americans say they would only vote...,/2019/06/06/politics/cnn-poll-abortion-importa...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190603101049-...,2019-06-07,2019,6,7,19,7,6,politics,"[<div class=""el__leafmedia el__leafmedia--sour..."
6,<strong>Duchess Meghan to make her first publi...,"Meghan Markle, Duchess of Sussex, will be maki...",/2019/06/07/europe/megan-markle-first-appearan...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190502113345-...,2019-06-07,2019,6,7,19,7,1,europe,"[<div class=""el__leafmedia el__leafmedia--sour..."
7,FedEx dumps Amazon from air cargo service as r...,Amazon packages will take their last US flight...,/2019/06/07/tech/fedex-amazon/index.html,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/160805083246-...,2019-06-07,2019,6,7,19,7,2,tech,"[<div class=""el__leafmedia el__leafmedia--sour..."
8,Body identified as missing Mouseketeer,A body that was found in an Oregon home two mo...,/2019/06/07/us/mouseketeer-dennis-day-dead/ind...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190607130139-...,2019-06-07,2019,6,7,19,7,3,us,"[<div class=""el__leafmedia el__leafmedia--sour..."
9,She killed her abusive husband. Now she's been...,A British woman who served almost 10 years in ...,/2019/06/07/europe/uk-sally-challen-intl-scli-...,NaN,NaN,//cdn.cnn.com/cnnnext/dam/assets/190607092315-...,2019-06-07,2019,6,7,19,7,4,europe,"[<div class=""el__leafmedia el__leafmedia--sour..."
